# Controller #

Import all neccessary packages

In [1]:
import os
import pandas as pd
import numpy as np

from controllers.inference_controllers.sandbox import Sandbox
from controllers.inference_controllers.generator import Generator
from controllers.inference_controllers.optimiser import Optimiser

from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Define how the data is to be generated and what default inputs should be used

In [2]:
# data_params = {
#     'data_type': 'simulated_data',
#     'data_path': 'data',
#     'sigma': 1,
#     'model_select': 'log_gpm_alt_norm',
#     'noise_dist': 'gaussian',
#     'model': {
#         'model_params':{
#             'H': 10
#         },
#         'inference_params':{
#             'I_y': 0.1,
#             'I_z': 0.1,
#             'Q': 3e13,
#             'sigma': 1
#         },
#     },
#     'domain': {
#         'domain_select': 'cone_from_source_z_limited', 
#         'resolution': 20,
#         'domain_params':{
#             'r': 100,
#             'theta': np.pi/8,
#             'source': [0,0,10]}
#     },
#     'output_header': 'Concentration',
#     'grid_dummy': True,
#     'grid_size': [10, 10, 10]
# }

data_params = {
    'data_type': 'normalised_data',
    'data_select': 'GBR_data',
    'normaliser_select': 'GBR_normaliser',
    'normaliser_params':{
        'experiments_list': [
            'Exp1',
            'Exp2',
            'Exp3',
            'Exp4',
            'Exp5',
            'Exp6',
            'Exp7',
            'Exp8',
            'Exp9',
            'Exp10',
            'Exp11',
            'Exp12',
        ],
        'meta_data_select': 'GBR_data_summary',
        'input_header': 'Concentration'
    },
    'log':True,
    'output_header': 'Concentration',
    'gridding': [100,100,25]
}

# data_params = {
#     'data_type': 'gridded',
#     'output_header': 'Concentration',
#     'log':True,
#     'grid_size': [100,100,25],
#     'target': False,
#     'data_path':'data'
# }


default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y': Parameter('I_y', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'I_z': Parameter('I_z', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'Q': Parameter('Q', prior_select = 'gamma', default_value=3e13).add_prior_param('mu', 3e13).add_prior_param('sigma',1e13),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'gamma', default_value=1).add_prior_param('mu', 1).add_prior_param('sigma',1)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

results_name = 'gridded_simulated_data_10_10_10'

## Sandbox ##

Create an instance with the inputted default parameters and visualise it in different ways

In [3]:
sandbox = Sandbox(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

TypeError: 'NoneType' object is not callable

## Generator ##

Generate different instances by varying the hyperparameters with respect to their defaults

In [ ]:
generator = Generator(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

Vary one hyperparameter

In [ ]:
# analysis_iterations = {
#     'parameters':
#     [
#         'sigma_mu', 
#         'sigma_sigma'
#     ],
#     'values':
#     [
#         np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]),
#         np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]),
#     ]
# }

# for i in range(len(analysis_iterations['parameters'])):
#     parameter = analysis_iterations['parameters'][i]
#     print('Working on varying ' + parameter + '...')
#     values = analysis_iterations['values'][i]
#     inputs = generator.vary_one_parameter(parameter, values, xscale='linear', plot=True)

Vary two hyperparameters

In [ ]:
analysis_iterations = {
    'parameters_1':
    [
        'I_y_mu',
        'I_z_mu',
        'Q_mu',
    ],
    'parameters_2':
    [
        'I_y_sigma',
        'I_z_sigma',
        'Q_sigma',

    ],
    'values_1':
    [
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e9, 1e10, 1e11, 1e12, 1e13, 1e14, 1e15, 1e16, 1e17, 1e18]),    ],
    'values_2':
    [
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e9, 1e10, 1e11, 1e12, 1e13, 1e14, 1e15, 1e16, 1e17, 1e18]),    ]
}

for i in range(len(analysis_iterations['parameters_1'])):
    parameter_1 = analysis_iterations['parameters_1'][i]
    parameter_2 = analysis_iterations['parameters_2'][i]

    print('Working on varying ' + parameter_1 + '...')
    values_1 = analysis_iterations['values_1'][i]
    values_2 = analysis_iterations['values_2'][i]
    inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1='log', scale_2='log', plot=True)

Working on varying I_y_mu...


Working on varying I_z_mu...
Generating instance 1...
Generating instance 2...
Generating instance 3...
Generating instance 4...
Generating instance 5...
Generating instance 6...
Generating instance 7...
Generating instance 8...
Generating instance 9...
Generating instance 10...
Generating instance 11...
Generating instance 12...
Generating instance 13...
Generating instance 14...
Generating instance 15...
Generating instance 16...
Generating instance 17...
Generating instance 18...
Generating instance 19...
Generating instance 20...
Generating instance 21...
Generating instance 22...
Generating instance 23...
Generating instance 24...
Generating instance 25...
Generating instance 26...
Generating instance 27...
Generating instance 28...
Generating instance 29...
Generating instance 30...
Generating instance 31...
Generating instance 32...
Generating instance 33...
Generating instance 34...
Generating instance 35...
Generating instance 36...
Generating instance 37...
Generating instanc

sample: 100%|██████████| 12500/12500 [00:12<00:00, 1037.97it/s, 3 steps of size 5.05e-01. acc. prob=0.92]


Generating instance 43...


sample: 100%|██████████| 12500/12500 [00:08<00:00, 1422.05it/s, 3 steps of size 5.02e-01. acc. prob=0.92] 


Generating instance 44...


sample: 100%|██████████| 12500/12500 [00:10<00:00, 1196.08it/s, 3 steps of size 5.01e-01. acc. prob=0.92]


Generating instance 45...


sample: 100%|██████████| 12500/12500 [00:11<00:00, 1082.61it/s, 3 steps of size 5.04e-01. acc. prob=0.93] 


## Optimiser ##

Optimise the default construction using Bayesian Optimisation

In [ ]:
optimising_parameters = {
                    'I_y_mu': [1e-2, 10],
                    'I_y_sigma': [1e-2, 10],
                    'I_z_mu': [1e-2, 10],
                    'I_z_sigma': [1e-2, 10],
                    'Q_mu': [1e9, 1e18],
                    'Q_sigma': [1e9, 1e18],
                    'sigma_mu':[0.2, 2],
                    'sigma_sigma':[0.2, 2]
                }

optimiser = Optimiser(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

study = optimiser.run(n_trials=100, optimiser_name='AIC_1', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)

optimiser.run_best_params(study)

[I 2023-07-28 12:08:28,287] A new study created in memory with name: no-name-35bdb3a9-5e00-40d8-9472-96212be3c101
sample: 100%|██████████| 12500/12500 [00:08<00:00, 1393.03it/s, 3 steps of size 5.16e-01. acc. prob=0.91]
[I 2023-07-28 12:08:59,760] Trial 0 finished with value: 283.00164794921875 and parameters: {'I_y_mu': 4.712087753026529, 'I_y_sigma': 8.835988871637127, 'I_z_mu': 3.6895512187907555, 'I_z_sigma': 1.3774047446847253, 'Q_mu': 9.003436586367254e+17, 'Q_sigma': 7.630440437777642e+17, 'sigma_mu': 1.561518988016868, 'sigma_sigma': 0.6087030381021383}. Best is trial 0 with value: 283.00164794921875.
sample: 100%|██████████| 12500/12500 [00:05<00:00, 2231.56it/s, 15 steps of size 6.05e-01. acc. prob=0.88]
[I 2023-07-28 12:09:24,600] Trial 1 finished with value: 282.70867919921875 and parameters: {'I_y_mu': 3.6899628934401996, 'I_y_sigma': 1.8347872013269941, 'I_z_mu': 7.165041358375334, 'I_z_sigma': 1.8040397007651363, 'Q_mu': 4.0297602291266086e+17, 'Q_sigma': 2.2709163065220

: 

: 